This notebook has examples of how to use the CDS API to do common queries and return results that should be usable, and exportable to a Cloud Resource. A few useful links:

    The CDS Data Model: https://dataservice.datacommons.cancer.gov/#/resources
    The CDS GraphQL endpoint: https://dataservice.datacommons.cancer.gov/v1/graphql/
    The GraphiQL interface in CDS (a good place to build queries): https://dataservice.datacommons.cancer.gov/#/graphql

Import a few useful libraries. Requests for carrying out the communications and Pandas for manipulating the returns. We also need json to help with formatting the results.


In [12]:
import pandas as pd
import requests

In [4]:
cds_graphql_url = "https://dataservice.datacommons.cancer.gov/v1/graphql/"

Since there will be multiple queries, having a simple routine that runs the query and returns the answer as a JSON object will streamline the notebook

In [9]:
def runGraphQLQuery(url,query):
    headers = {"content-type":"application/json"}
    try:
        results = requests.post(url = url, json={"query":query, "headers":headers})
        results = results.json()
    except requests.exceptions.HTTPError as exception:
        print(exception)
    return results

CDS has multiple studies, so as a first step, it's worth taking a look to see what content is avaialble.  This first query simply lists out the different studies that have data (This notebook assumes that you've done some basic introspection queries to get familiar wtih the content of the CDS database).

In [10]:
study_query = """
{
  study{
    study_name
    study_description
    study_acronym
    study_access
    phs_accession
  }
}
"""

This query will bring back several useful pieces of information that should help decide what studies are worth further examination.  You can also run this query in the GraphiQL interface in the CDS portal (https://dataservice.datacommons.cancer.gov/#/graphql) to see what the JSON object looks like.

In [13]:
study_result = runGraphQLQuery(cds_graphql_url, study_query)

Let's put the information in a dataframe so we can easily make it more readable than a JSON object

In [19]:
df_columns = ['Study Name', 'Acronym', 'Access', 'Accession', 'Description']
study_df = pd.DataFrame(columns = df_columns)
for study in study_result['data']['study']:
    study_df.loc[len(study_df.index)] = [study['study_name'],study['study_acronym'],study['study_access'], study['phs_accession'],study['study_description']]

In [22]:
pd.set_option('display.max_colwidth', None)
study_df

,Study Name,Acronym,Access,Accession,Description
0,Clinical Trial Specimen Molecular Characterization (CTSMC),CTSMC,Controlled,phs002790,"The National Cancer Institute's (NCI) Childhood Cancer Data Initiative (CCDI) focuses on the critical need to collect, analyze, and share data to address the burden of cancer in children, adolescents, and young adults (AYAs). The Molecular Characterization Initiative (MCI) will further the CCDI's goals by providing access to better diagnostic tests for pediatric and AYA patients. The molecular characterizations of solid tumors, soft tissue sarcomas, and rare diseases are performed in a CLIA-certified setting as results may be used to screen for and/or confirm clinical trial eligibility, direct treatment, or otherwise contribute to the conduct of the trial. The following molecular characterization assays were performed:\n\nTumor/Normal Whole Exome sequencing\nMethylation arrays\nArcher fusion panel\nDeidentified clinical reports\nNote: New data will be added periodically, please check the SBG site for updates.\n\nStudy Weblinks:\nCCDI Molecular Characterization Initiative\nStudy Design:\nTumor vs. Matched-Normal\nStudy Type:\nAggregate Genomic Data\nClinical Diagnostic Testing\nExome Sequencing\nIndividual-Level Genomic Data\nTranscriptome Analysis\nTumor vs. Matched-Normal\nTotal number of consented subjects: 1996"
1,"UCSF Database for the Advancement of JMML - Integration of Metadata with """"Omic"""" Data",JMML,Controlled,phs002504,"Juvenile myelomonocytic leukemia (JMML) is a rare and frequently fatal myeloproliferative/myelodysplastic disorder of early childhood with an estimated incidence of 1.2 cases per million. It is associated with a spectrum of diverse outcomes ranging from spontaneous resolution in rare patients to transformation to acute myeloid leukemia in others. The overwhelming majority of JMML patients (~95%) will harbor mutations in canonical Ras pathway genes, including NF1, NRAS, KRAS, PTPN11, and CBL. As Ras proteins are mutated in more than 30% of human cancers, the information gleaned from the study of JMML has provided insights into Ras signaling in cancer as well as a group of congenital diseases with tumor predispositions known as the “Rasopathies”. While oncogenic Ras is one of the most common mutations in human cancer, it remains one of the most vexing targets for efficacious therapy."
2,Human Tumor Atlas Network (HTAN) primary sequencing data,HTAN,Controlled,phs002371,"An NCI-funded Cancer Moonshot initiative to construct 3-dimensional atlases of the dynamic cellular, morphological, and molecular features of human cancers as they evolve from precancerous lesions to advanced disease."
3,"CIDR: Discovery, Biology, and Risk of Inherited Variants in Glioma sample",None,Controlled,phs002250,"This is a gliogene brain tumor family study. This study includes glioma cases with a family history of glioma in a first, second, or third degree relative. \nStudy Weblinks:\nGliogene\nStudy Design:\nFamily/Twin/Trios\nStudy Type:\nFamily\nWhole Genome Sequencing\ndbGaP estimated ancestry using GRAF-pop\nTotal number of consented subjects: 151"
4,Childhood Cancer Data Initiative (CCDI): Free the Data: Open Sharing of Comprehensive Genomic Childhood Cancer Datasets (Kansas),CCDI-KUMC,Controlled,phs002529,"This study provides paired tumor normal genomic sequencing data from approximately 200 children with cancer, including both solid tumors and leukemias, done by the Children’s Mercy Research Institute (CMRI) and University of Kansas Cancer Center (KUCC).\n\nThese data include whole genome sequencing (generally, ~20x), whole exome sequencing (generally, ~300x), bulk RNA sequencing (generally, ~80 million reads), and single-cell RNA and ATAC sequencing (>50,000 reads/cell). Additional phenotypic, pathologic, and genetic data, gathered clinically for these samples, are also provided.\nStudy Design:\n Tumor vs. Matched-Normal\nStudy Type:\nCase Set\nClinical Cohort\nClinical Genetic Te

From this list, HTAN looks interesting, let's take a look at some numbers to see if there's enough data there to support further analysis

In [27]:
info_query= """
{
  studyDetail(phs_accession:"phs001287"){
    data_types
    numberOfFiles
    numberOfSubjects
    numberOfSamples
  }
}
"""

In [28]:
info_results = runGraphQLQuery(cds_graphql_url, info_query)

In [29]:
print(info_results)

{'data': {'studyDetail': {'data_types': 'Genomic', 'numberOfFiles': 1243, 'numberOfSubjects': 1069, 'numberOfSamples': 1114}}}
